In [ ]:

data5min = RandomOHLCV( 
    freq      = '5 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2000',           
    open_val  = 100.00,           
    periods   = 1_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
)

data.get_dataframe().head(10))

f5min = Frame(symbol='TSLA')

# TA may add many several columns of data to the dataframe 
f5min.add_ta(MA('close', 9),  {'line_style': 'solid', 'colour': 'red', 'thickness': 2}) # creates new column to the dataframe with the moving average eg MA_C9
f5min.add_ta(MA('close', 21), {'line_style': 'solid', 'colour': 'blue', 'thickness': 2})
f5min.add_ta(MA('close', 28), {'line_style': 'solid', 'colour': 'green', 'thickness': 2})
f5min.add_ta(MACD(fast=('close', 12), slow=('close', 26), signal=('close', 9), faststyle={'colour': 'red'}, slowstyle={'colour': 'blue'}, signalstyle={'colour': 'green', 'thickness': 3})) # creates new columns to the dataframe with the MACD values. eg  looks for the MA_9 and MA_21 columns to calculate the MACD values if not then it will call MA(9) and MA(21) to calculate the MACD values


# all strategies return a single value between 0 and 100 .  signals have their own dataframe and one column for each signal
f5min.add_signal(NearMA(MA('close', 9), 0.1)).score(normRange=(0, 100)) # creates new column to the dataframe with the signal values. eg 1 if the close price is near the MA(9) by 0.1
f5min.add_signal(Tail(0.5)).score(normRange=(0, 100)) 
f5min.add_signal(SmoothPullback(0.5)).score(normRange=(0, 100))
f5min.add_signal(ChangeOfColour()).score(normRange=(0, 100))

f5min.add_entry_strategy(PullbackSetup(
    times              = [('00:01', '23:59')],
    barSizes           = ['1 D'], 
    ls                 = ['LONG', 'SHORT'],
    minAvPass=3, maxFail=1, minAvScore=50, 
    signals=[
        ('NearMA_C9', passRange=(0, 100), weigth=1),
        ('Tail_50', passRange=(0, 100), weigth=1),
        ('SmoothPullback_50', passRange=(0, 100), weigth=1),
        ('ChangeOfColour', passRange=(0, 100), weigth=1)]
    ))

f5min.plot()



bot = StockBot(mode='backtest', capital=1000, risk=0.02, riskType='percent', commission=0.01, slippage=0.01)


# run for every new row of data.  used for both back testing and live trading
# new data is updated row wise so as not to recalculate for previous rows where possible.  be careful of MA as this requires several past rows to calculate
f5min.update_ohlcv({'open': 1.0, 'high': 1.1, 'low': 0.9, 'close': 1.0, 'volume': 1000}).update_ta().update_signals()
f60min.update_ohlcv({'open': 1.0, 'high': 1.1, 'low': 0.9, 'close': 1.0, 'volume': 1000}).update_ta().update_signals()
f1day.update_ohlcv({'open': 1.0, 'high': 1.1, 'low': 0.9, 'close': 1.0, 'volume': 1000}).update_ta().update_signals()
market.update_ohlcv({'open': 1.0, 'high': 1.1, 'low': 0.9, 'close': 1.0, 'volume': 1000})

bot.run(f2min, f5min, f60min, f1day, market).show_reslts()
bot.market_bias()
bot.stock_bias()
bot.has_room_to_move()
bot.has_capital()
bot.has_position()

# Needs a bit more thoguht on how to implement this
bot.trade(ExitStrategy(stops=[
    Stop(implimentWhenRR=True, stop='prior_piv', offset=0.01),
    Stop(implimentWhenRR=2, stop='bar_by_bar', offset=0.01)], 
    target=Target(riskReward=2))

bot.show_trade_progress()


# Example usage of the Stockbot system. 

In [ ]:

# Run a pre market scan

# Assess the results from the pre market scan looking for stock bias and market bias, And high probability is successful trades
# cheque for room to move and gaps etc. 

# Give the results to Stcokbot to run the trades, stratgeies and signals, manage accounts and trades


: 